# Actividad M1
_Oscar Miranda Escalante_
_A01630791_

### Código de la solución
Este código ejecuta la simulación y hace un seguimiento del número de 'robots'
que limpian, el porcentaje de celdas que se han limpiado, tiempo y número de
movimientos realizados en total por todos los agentes.

Repositorio en Github: https://github.com/oscarmires/agentpy-projects/blob/master/m1.ipynb

In [3]:
import agentpy as ap
import seaborn as sns


class Room(ap.Model):

    def setup(self):
        # Initialize counters
        self.step_count = 0
        self.total_movements = 0

        # Initialize agents
        #   Cleaners
        n_cleaners = self.p.cleaners
        self.cleaners = ap.AgentList(self, n_cleaners)
        #   Tiles
        n_tiles = self.n_tiles = self.p.m * self.p.n
        self.tiles = ap.AgentList(self, n_tiles)
        #   Dirty tiles
        n_dirty = self.n_initially_dirty = int(n_tiles * self.p.dirt)

        # Set conditions (0: clean, 1: cleaning, 2: dirty)
        self.tiles.condition = 0
        self.tiles.isTile = True
        for i in range(n_dirty):
            self.tiles[i].condition = 2
        self.cleaners.target = None
        self.cleaners.isTile = False

        # Create grid
        self.grid = ap.Grid(self, (self.p.m, self.p.n), track_empty=True)
        self.grid.add_agents(self.tiles, random=True, empty=True)
        self.grid.add_agents(self.cleaners, [(1, 1)] * n_cleaners)

    def step(self):
        for cleaner in self.cleaners:
            if cleaner.target:
                target = cleaner.target
                if target.condition == 1:
                    target.condition = 0
                    self.grid.move_to(cleaner, self.grid.positions[target])
                    self.total_movements += 1
                    cleaner.target = None
            else:
                neighbors = self.grid.neighbors(cleaner)
                n_neighbor = len(neighbors)
                rand_pos = self.random.randint(0, n_neighbor - 1)
                neighbor = neighbors.to_list()[rand_pos]

                if neighbor.isTile:
                    if neighbor.condition == 2:
                        cleaner.target = neighbor
                        neighbor.condition = 1
                    elif neighbor.condition == 0:
                        self.grid.move_to(cleaner, self.grid.positions[neighbor])
                        self.total_movements += 1

        self.step_count += 1

        clean = self.tiles.select(self.tiles.condition == 0)
        allClean = len(clean) == self.n_tiles

        if self.step_count == self.p.steps or allClean:
            self.stop()

    def end(self):
        dirty_tiles = len(self.tiles.select(self.tiles.condition == 2))
        self.report('cleaner_agents', len(self.cleaners))
        self.report('percentage_cleaned', f'{(self.n_initially_dirty - dirty_tiles) / self.n_initially_dirty * 100}%')
        self.report('time', f'{self.step_count}/{self.p.steps}')
        self.report('movements', self.total_movements)


parameters = {
    'dirt': 0.2,  # probability of dirty cells
    'm': 10,  # grid's l
    'n': 10,  # grid's h
    'steps': 500,  # time
    'cleaners': 100
}

# Configure experiment
parameters_multi = dict(parameters)
parameters_multi.update({
    'steps': ap.Values(50, 100, 500),  # time
    'cleaners': ap.Values(1, 5, 20)
})
sample = ap.Sample(parameters_multi)

exp = ap.Experiment(Room, sample, iterations=1)
results = exp.run()

# print results
results
columns = ['cleaner_agents',
           'percentage_cleaned',
           'time',
           'movements']
print(results['reporters'][columns])

Scheduled runs: 9
Completed: 9, estimated time remaining: 0:00:00
Experiment finished
Run time: 0:00:00.361655
           cleaner_agents percentage_cleaned     time  movements
sample_id                                                       
0                       1              35.0%    50/50         43
1                       5              70.0%    50/50        220
2                      20              90.0%    50/50        714
3                       1              25.0%  100/100         95
4                       5              95.0%  100/100        455
5                      20             100.0%   73/100        979
6                       1              95.0%  500/500        481
7                       5             100.0%  194/500        898
8                      20             100.0%   88/500       1108


## Análisis
Resumen mejor resultado en el experimento (20 agentes limpiadores)

| Porcentaje de celdas sucias que se limpiaron | Tiempo necesario | Cantidad de movimientos |
|---|---|---|
| 100% | 88 | 1108 |

En los resultados que imprime el código, se observa que conforme hay más agentes limpiadores,
aumenta el porcentaje de celdas limpias y la cantidad de movimientos.